In [10]:
import pandas as pd
import os
import tqdm

datos_path = 'C:/Users/santi/GNN4OPF/'

### Para generar el csv con los 3 años concatenados

In [38]:
GEN = pd.read_excel(datos_path+'GEN.xlsx', sheet_name='2021')

In [39]:
GEN2 = pd.read_excel(datos_path+'GEN.xlsx', sheet_name='2022')
GEN3 = pd.read_excel(datos_path+'GEN.xlsx', sheet_name='2023')

In [40]:
df_concatenado = pd.concat([GEN, GEN2, GEN3])

In [ ]:
df_concatenado.to_csv(datos_path + 'GEN_total.csv', index=False)

In [3]:
GEN_loaded = pd.read_csv(datos_path+'GEN_total.csv')

C:\Users\santi\AppData\Local\Temp\ipykernel_33412\81847354.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,

In [9]:
GEN_loaded['Fecha']

dtype('O')

### Funcion para levantar las columnas con los nodos que querramos, y guardar en un csv con los nombres de los buses

In [40]:
def create_dataset(path, buses, out_dir=None):
    original_data = pd.read_csv(path)
    # chequeo de colunmas
    for bus in buses:
        if bus not in original_data.columns:
            raise ValueError(f'Column {bus} not found in the dataset')
    clean_data = pd.DataFrame(columns=buses.keys())
    clean_data['Fecha'] = pd.to_datetime(clean_data['Fecha'])
    for bus in tqdm.tqdm(buses, desc='Cleaning data',total=len(buses)):
        if bus != 'Fecha':
            clean_data[bus] = original_data[bus]
            clean_data = clean_data.rename(columns={bus: buses[bus]})
            clean_data = clean_data.applymap(convertir_a_cero)
    if 'Fecha' in buses:
        clean_data['Fecha'] = original_data['Fecha']
    if out_dir:
        clean_data.to_csv(out_dir, index=False)

    return clean_data

def convertir_a_cero(elemento):
    if isinstance(elemento, str):
        return 0
    else:
        return elemento

In [26]:
buses_pgen = {'Fecha': 'Fecha',
'SGA500.013 GE G01 MED P.AV': '4000_1',
'SGA500.013 GE G02 MED P.AV': '4000_2',
'SGA500.013 GE G03 MED P.AV': '4000_3',
'SGA500.013 GE G04 MED P.AV': '4000_4',
'SGA500.013 GE G05 MED P.AV': '4000_5',
'SGA500.013 GE G06 MED P.AV': '4000_6',
'SGA500.013 GE G13 MED P.AV': '4000_7',
'SGU500.013 GE G07 MED P.AV': '90000_1',
'SGU500.013 GE G08 MED P.AV': '90000_2',
'SGU500.013 GE G09 MED P.AV': '90000_3',
'SGU500.013 GE G10 MED P.AV': '90000_4',
'SGU500.013 GE G11 MED P.AV': '90000_5',
'SGU500.013 GE G12 MED P.AV': '90000_6',
'SGU500.013 GE G14 MED P.AV': '90000_7',
'PTIGRE.011 GE G01 MED P.AV': '90160_1',
'PTIGRE.011 GE G02 MED P.AV': '90160_2',
'PTIGRE.011 GE G03 MED P.AV': '90160_3',
'PTIGRE.011 GE G04 MED P.AV': '90160_4',
'PTIGRE.011 GE G05 MED P.AV': '90160_5',
'PTIGRE.011 GE G06 MED P.AV': '90160_6',
'PTIGRE.150 GE PTI APRC MED P.AV': '90160_7',
'PTIGREB.150 GE STG MED P.AV': '90160_8',
'PTIGRECC.150 GE GST MED P.AV':'90160_9' ,
'PTIGRECC.150 GE GT1 MED P.AV':'90160_10' ,
'PTIGRECC.150 GE GT2 MED P.AV':'90160_11' ,
'EKIYU.030 GE TOT MED P.AV': '90160_12',
'CENPAL.500 GE G01 MED P.AV': '90500_1',
'CENPAL.500 GE G02 MED P.AV': '90500_2',
'CENPAL.500 GE G03 MED P.AV': '90500_3',
'NARANJAL.150 GE TOT MED P.AV':'92000_1' ,
'DLITORAL.030 GE TOT MED P.AV':'92040_1' ,
'PALOMAS.030 GE TOT MED P.AV': '92061_1',
'JPTERRA.030 GE TOT MED P.AV': '92091_1',
'MVL150.150 GE CT1 MED P.AV': '92124_1',
'MVL150.150 GE CT2 MED P.AV': '92124_2',
'CBMOT.011 GE G01 MED P.AV': '92240_1',
'CBMOT.011 GE G02 MED P.AV': '92240_2',
'CBMOT.011 GE G03 MED P.AV': '92240_3',
'CBMOT.011 GE G04 MED P.AV': '92240_4',
'CBMOT.011 GE G05 MED P.AV': '92240_5',
'CBMOT.011 GE G06 MED P.AV': '92240_6',
'CBMOT.011 GE G07 MED P.AV': '92240_7',
'CBMOT.011 GE G08 MED P.AV': '92240_8',
'BIOENER.030 GE TOT MED P.AV': '92400_1',
'PONLAR.030 GE TOT MED P.AV': '92400_2',
'EPAMPA.030 GE TOT MED P.AV': '92411_1',
'EVALENTI.030 GE TOT MED P.AV':'92420_1' ,
'PCGRANDE.030 GE TOT MED P.AV':'92485_1' ,
'MELOWIND.030 GE TOT MED P.AV':'92490_1' ,
'EFLOR1.030 GE TOT MED P.AV': '92524_1',
'EFLOR2.030 GE TOT MED P.AV': '92524_2',
'BONETE.013 GE TOT MED P.AV': '92540_1',
'BAYGORRI.150 GE G01 MED P.AV':'92550_1' ,
'BAYGORRI.150 GE G02 MED P.AV':'92550_2' ,
'BAYGORRI.150 GE G03 MED P.AV':'92550_3' ,
'TMACIEL1.030 GE TOT MED P.AV':'92590_1' ,
'TMACIEL2.030 GE TOT MED P.AV':'92590_2' ,
'ECARIAS.030 GE TOT MED P.AV': '92590_3',
'ENPASTO1.030 GE TOT MED P.AV':'92590_4' ,
'ARTILLER.150 GE TOT MED P.AV':'92611_1' ,
'RDELSUR.030 GE TOT MED P.AV': '92700_1',
'EMALD2.030 GE TOT MED P.AV': '92700_2',
'MALDONAD.030 GE G01 MED P.AV':'92700_3' ,
'CARACOLE.030 GE TOT MED P.AV':'92700_4' ,
'EMINAS1.030 GE TOT MED P.AV': '92710_1',
'ECARAPE1.030 GE TOT MED P.AV':'92760_1' ,
'ECARAPE2.030 GE TOT MED P.AV':'92760_2' ,
'PERALTA1.030 GE TOT MED P.AV':'92790_1' ,
'PERALTA2.030 GE TOT MED P.AV':'92790_2' ,
'PALMATIR.030 GE TOT MED P.AV':'92790_3' ,
'LUZLOMA.030 GE TOT MED P.AV': '92910_1',
'LUZMAR.030 GE TOT MED P.AV': '92910_2',
'LUZRIO.030 GE TOT MED P.AV': '92910_3'}

In [27]:
create_dataset(datos_path+'GEN_total.csv', buses_pgen,datos_path + 'GEN_total_psse.csv')

C:\Users\santi\AppData\Local\Temp\ipykernel_33412\3137351998.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,25

,Fecha,4000_1,4000_2,4000_3,4000_4,4000_5,4000_6,4000_7,90000_1,90000_2,...,92700_4,92710_1,92760_1,92760_2,92790_1,92790_2,92790_3,92910_1,92910_2,92910_3
0,2021-07-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
1,2021-07-01 00:10:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
2,2021-07-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
3,2021-07-01 00:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
4,2021-07-01 00:40:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135934,2024-01-30 23:20:00,130.000000,130.800003,130.600006,130.600006,129.199997,130.900009,129.600006,127.900002,128.000000,...,4.723,26.436001,9.480,9.264,1.9845,0.0000,1.944,2.3,1.7,4.6
135935,2024-01-30 23:30:00,129.300003,131.900009,130.300003,130.199997,129.400009,130.699997,129.600006,128.600006,127.900002,...,2.815,28.188000,10.728,9.828,1.9845,0.0000,1.392,2.3,1.7,4.0
135936,2024-01-30 23:40:00,128.900009,132.300003,130.000000,130.199997,129.100006,130.000000,129.600006,128.699997,128.199997,...,1.653,26.382000,9.828,11.226,1.9845,0.0000,1.392,2.3,1.1,3.3
135937,2024-01-30 23:50:00,129.500000,131.000000,130.800003,130.199997,129.300003,130.800003,129.600006,127.900002,128.199997,...,2.760,28.278000,11.292,11.160,2.6460,1.9845,1.908,2.3,1.1,3.3


In [45]:
entradas = [
    ("SGA500.013 GE G01 MED Q.AV", 4000),
    ("SGA500.013 GE G02 MED Q.AV", 4000),
    ("SGA500.013 GE G03 MED Q.AV", 4000),
    ("SGA500.013 GE G04 MED Q.AV", 4000),
    ("SGA500.013 GE G05 MED Q.AV", 4000),
    ("SGA500.013 GE G06 MED Q.AV", 4000),
    ("SGA500.013 GE G13 MED Q.AV", 4000),
    ("SGU500.013 GE G07 MED Q.AV", 90000),
    ("SGU500.013 GE G08 MED Q.AV", 90000),
    ("SGU500.013 GE G09 MED Q.AV", 90000),
    ("SGU500.013 GE G10 MED Q.AV", 90000),
    ("SGU500.013 GE G11 MED Q.AV", 90000),
    ("SGU500.013 GE G12 MED Q.AV", 90000),
    ("SGU500.013 GE G14 MED Q.AV", 90000),
    ("PTIGRE.011 GE G01 MED Q.AV", 90160),
    ("PTIGRE.011 GE G02 MED Q.AV", 90160),
    ("PTIGRE.011 GE G03 MED Q.AV", 90160),
    ("PTIGRE.011 GE G04 MED Q.AV", 90160),
    ("PTIGRE.011 GE G05 MED Q.AV", 90160),
    ("PTIGRE.011 GE G06 MED Q.AV", 90160),
    # (None, 90160),  # Supondremos que queremos omitir esta entrada
    ("PTIGREB.150 GE STG MED Q.AV", 90160),
    ("PTIGRECC.150 GE GST MED Q.AV", 90160),
    ("PTIGRECC.150 GE GT1 MED Q.AV", 90160),
    ("PTIGRECC.150 GE GT2 MED Q.AV", 90160),
    ("EKIYU.030 GE EOL MED Q.AV", 90160),
    ("CENPAL.500 GE G01 MED Q.AV", 90500),
    ("CENPAL.500 GE G02 MED Q.AV", 90500),
    ("CENPAL.500 GE G03 MED Q.AV", 90500),
    ("NARANJAL.030 GE SOL MED Q.AV", 92000),
    # ("SE BORRA", 92000),
    ("DLITORAL.030 GE SOL MED Q.AV", 92040),
    ("PALOMAS.030 GE EOL MED Q.AV", 92061),
    ("JPTERRA.030 GE EJT MED Q.AV", 92091),
    ("MVL150.150 GE CT1 MED Q.AV", 92124),
    ("MVL150.150 GE CT2 MED Q.AV", 92124),
    ("CBMOT.011 GE G01 MED Q.AV", 92240),
    ("CBMOT.011 GE G02 MED Q.AV", 92240),
    ("CBMOT.011 GE G03 MED Q.AV", 92240),
    ("CBMOT.011 GE G04 MED Q.AV", 92240),
    ("CBMOT.011 GE G05 MED Q.AV", 92240),
    ("CBMOT.011 GE G06 MED Q.AV", 92240),
    ("CBMOT.011 GE G07 MED Q.AV", 92240),
    ("CBMOT.011 GE G08 MED Q.AV", 92240),
    # ("SE BORRA", 92400),
    ("BIOENER.030 GE BIO MED Q.AV", 92400),
    # ("Falta Q", 92400),
    ("EPAMPA.150 GE MOL MED Q.AV", 92411),
    ("EVALENTI.030 GE EOL MED Q.AV", 92420),
    ("PCGRANDE.030 GE EOL MED Q.AV", 92485),
    ("MELOWIND.030 GE EOL MED Q.AV", 92490),
    ("EFLOR1.030 GE EOL MED Q.AV", 92524),
    ("EFLOR2.030 GE EOL MED Q.AV", 92524),
    ("BONETE.013 GE TOT MED Q.AV", 92540),
    # ("SE BORRA", 92540),
    # ("SE BORRA", 92540),
    # ("SE BORRA", 92540),
    ("BAYGORRI.150 GE G01 MED Q.AV", 92550),
    ("BAYGORRI.150 GE G02 MED Q.AV", 92550),
    ("BAYGORRI.150 GE G03 MED Q.AV", 92550),
    ("TMACIEL1.030 GE EOL MED Q.AV", 92590),
    ("TMACIEL2.030 GE EOL MED Q.AV", 92590),
    ("ECARIAS.030 GE EOL MED Q.AV", 92590),
    ("ENPASTO1.030 GE EOL MED Q.AV", 92590),
    ("ARTILLER.030 GE EOL MED Q.AV", 92611),
    ("RDELSUR.030 GE EOL MED Q.AV", 92700),
    ("EMALD2.030 GE EOL MED Q.AV", 92700),
    ("MALDONAD.030 GE G01 MED Q.AV", 92700),
    # ("Falta Q", 92700),
    ("EMINAS1.030 GE EOL MED Q.AV", 92710),
    ("ECARAPE1.030 GE EOL MED Q.AV", 92760),
    ("ECARAPE2.030 GE EOL MED Q.AV", 92760),
    ("PERALTA1.030 GE EOL MED Q.AV", 92790),
    ("PERALTA2.030 GE EOL MED Q.AV", 92790),
    ("PALMATIR.000 GE EOL MED Q.AV", 92790)
    # ("Falta q", 92910),
    # ("Falta q", 92910),
    # ("Falta q", 92910)
]

# Creación del diccionario con manejo de claves duplicadas
diccionario = {}
dicc2 = {}
for valor, clave in entradas:
    original = clave
    i = 1
    clave = f"{original}_{i}"
    while clave in diccionario:
        i += 1
        clave = f"{original}_{i}"
    diccionario[clave] = valor
    dicc2[valor] = clave

dicc2['Fecha'] = 'Fecha'
# Mostrar el diccionario resultante
for k, v in dicc2.items():
    print(f"{k}: {v}")  


SGA500.013 GE G01 MED Q.AV: 4000_1
SGA500.013 GE G02 MED Q.AV: 4000_2
SGA500.013 GE G03 MED Q.AV: 4000_3
SGA500.013 GE G04 MED Q.AV: 4000_4
SGA500.013 GE G05 MED Q.AV: 4000_5
SGA500.013 GE G06 MED Q.AV: 4000_6
SGA500.013 GE G13 MED Q.AV: 4000_7
SGU500.013 GE G07 MED Q.AV: 90000_1
SGU500.013 GE G08 MED Q.AV: 90000_2
SGU500.013 GE G09 MED Q.AV: 90000_3
SGU500.013 GE G10 MED Q.AV: 90000_4
SGU500.013 GE G11 MED Q.AV: 90000_5
SGU500.013 GE G12 MED Q.AV: 90000_6
SGU500.013 GE G14 MED Q.AV: 90000_7
PTIGRE.011 GE G01 MED Q.AV: 90160_1
PTIGRE.011 GE G02 MED Q.AV: 90160_2
PTIGRE.011 GE G03 MED Q.AV: 90160_3
PTIGRE.011 GE G04 MED Q.AV: 90160_4
PTIGRE.011 GE G05 MED Q.AV: 90160_5
PTIGRE.011 GE G06 MED Q.AV: 90160_6
PTIGREB.150 GE STG MED Q.AV: 90160_7
PTIGRECC.150 GE GST MED Q.AV: 90160_8
PTIGRECC.150 GE GT1 MED Q.AV: 90160_9
PTIGRECC.150 GE GT2 MED Q.AV: 90160_10
EKIYU.030 GE EOL MED Q.AV: 90160_11
CENPAL.500 GE G01 MED Q.AV: 90500_1
CENPAL.500 GE G02 MED Q.AV: 90500_2
CENPAL.500 GE G03 MED Q.AV:

In [46]:
create_dataset(datos_path+'GEN_total.csv', dicc2, datos_path + 'GENQ_total_psse.csv')

C:\Users\santi\AppData\Local\Temp\ipykernel_33412\677009404.py:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255

,4000_1,4000_2,4000_3,4000_4,4000_5,4000_6,4000_7,90000_1,90000_2,90000_3,...,92700_1,92700_2,92700_3,92710_1,92760_1,92760_2,92790_1,92790_2,92790_3,Fecha
0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000,0.000000,0,0.00,0.0,0.0,0.00000,0.0,0.000000,2021-07-01 00:00:00
1,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000,0.000000,0,0.00,0.0,0.0,0.00000,0.0,0.000000,2021-07-01 00:10:00
2,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000,0.000000,0,0.00,0.0,0.0,0.00000,0.0,0.000000,2021-07-01 00:20:00
3,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000,0.000000,0,0.00,0.0,0.0,0.00000,0.0,0.000000,2021-07-01 00:30:00
4,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000,0.000000,0,0.00,0.0,0.0,0.00000,0.0,0.000000,2021-07-01 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135934,11.7,11.500000,11.200000,11.900001,11.3,12.200000,11.500000,11.500000,12.3,11.700000,...,-0.653,5.083614,0,-9.35,0.0,0.0,3.78000,0.0,1.029848,2024-01-30 23:20:00
135935,11.7,11.500000,10.900001,11.300000,11.2,11.800000,11.500000,10.700000,11.6,11.800000,...,-0.653,5.083614,0,-10.97,0.0,0.0,4.48875,0.0,1.029848,2024-01-30 23:30:00
135936,11.0,11.400001,10.900001,11.700000,11.2,11.800000,11.500000,10.900001,11.6,11.800000,...,-0.653,5.083614,0,-9.89,0.0,0.0,5.19750,0.0,1.029848,2024-01-30 23:40:00
135937,12.2,12.000000,11.400001,12.100000,11.2,12.400001,11.500000,11.100000,11.6,11.900001,...,-0.969,5.083614,0,-12.12,0.0,0.0,5.19750,0.0,1.029848,2024-01-30 23:50:00


In [47]:
entradas = [
    (92670, "ACORRIEN.000 ES CAR MED P.AV"),
    (92060, "ARAPEY.000 ES CAR MED P.AV"),
    (92090, "ARTIGAS.000 ES CAR MED P.AV"),
    (92710, "BIFURCAC.000 ES CAR MED P.AV"),
    (92540, "BONETE.000 ES CAR MED P.AV"),
    (92600, "COLONIA.000 ES CAR MED P.AV"),
    (92690, "CONCHILL.000 ES CAR MED P.AV"),
    (92940, "DOLORES.000 ES CAR MED P.AV"),
    (92531, "DURAZNO.000 ES CAR MED P.AV"),
    (92681, "EFICE.000 ES CAR MED P.AV"),
    (92440, "EMARTINE.000 ES CAR MED P.AV"),
    (92050, "FBENTOS.000 ES CAR MED P.AV"),
    (92520, "FLORIDA.000 ES CAR MED P.AV"),
    (92620, "LIBERTAD.000 ES CAR MED P.AV"),
    (92730, "MALDONAD.000 ES CAR MED P.AV"),
    (92450, "MELO.000 ES CAR MED P.AV"),
    (92640, "MERCEDES.000 ES CAR MED P.AV"),
    (92100, "MVA150.000 ES CAR MED P.AV"),
    (90200, "MVB150.000 ES CAR MED P.AV"),
    (92210, "MVC150.000 ES CAR MED P.AV"),
    (92240, "MVE150.000 ES CAR MED P.AV"),
    (92310, "MVF150.000 ES CAR MED P.AV"),
    (92320, "MVG150.000 ES CAR MED P.AV"),
    (92330, "MVH150.000 ES CAR MED P.AV"),
    (92270, "MVJ150.000 ES CAR MED P.AV"),
    (92340, "MVK150.000 ES CAR MED P.AV"),
    (92180, "MVM150.000 ES CAR MED P.AV"),
    (92350, "MVR150.000 ES CAR MED P.AV"),
    (92110, "NORTE.000 ES CAR MED P.AV"),
    (92650, "NPALMIRA.000 ES CAR MED P.AV"),
    (92500, "PAL150.000 ES CAR MED P.AV"),
    (92770, "PANDO.000 ES CAR MED P.AV"),
    (92030, "PAYSANDU.000 ES CAR MED P.AV"),
    (92721, "PAZUCAR.000 ES CAR MED P.AV"),
    (101,   "PESTE.000 ES CAR MED P.AV"),
    (92280, "PIEDRAS.000 ES CAR MED P.AV"),
    (92400, "RIVERA.000 ES CAR MED P.AV"),
    (92760, "ROCHA.000 ES CAR MED P.AV"),
    (92580, "RODRIGUE.000 ES CAR MED P.AV"),
    (92610, "ROSARIO.000 ES CAR MED P.AV"),
    (92040, "SALTO.000 ES CAR MED P.AV"),
    (92700, "SCA150.000 ES CAR MED P.AV"),
    (92380, "SOLYMAR.000 ES CAR MED P.AV"),
    (92470, "MDIAZ.000 ES CAR MED P.AV"),
    (92630, "SVAZQUEZ.000 ES CAR MED P.AV"),
    (92410, "TAC150.000 ES CAR MED P.AV"),
    (92070, "TGOMENSO.000 ES CAR MED P.AV"),
    (92590, "TRINIDAD.000 ES CAR MED P.AV"),
    (92430, "TYTRES.000 ES CAR MED P.AV"),
    (92420, "VALENTIN.000 ES CAR MED P.AV"),
    (92020, "YOUNG.000 ES CAR MED P.AV"),
]

# Creación del diccionario con manejo de claves duplicadas
diccionario = {j:i for i,j in entradas}

diccionario['Fecha'] = 'Fecha'
# Mostrar el diccionario resultante
for k, v in diccionario.items():
    print(f"{k}: {v}")  

ACORRIEN.000 ES CAR MED P.AV: 92670
ARAPEY.000 ES CAR MED P.AV: 92060
ARTIGAS.000 ES CAR MED P.AV: 92090
BIFURCAC.000 ES CAR MED P.AV: 92710
BONETE.000 ES CAR MED P.AV: 92540
COLONIA.000 ES CAR MED P.AV: 92600
CONCHILL.000 ES CAR MED P.AV: 92690
DOLORES.000 ES CAR MED P.AV: 92940
DURAZNO.000 ES CAR MED P.AV: 92531
EFICE.000 ES CAR MED P.AV: 92681
EMARTINE.000 ES CAR MED P.AV: 92440
FBENTOS.000 ES CAR MED P.AV: 92050
FLORIDA.000 ES CAR MED P.AV: 92520
LIBERTAD.000 ES CAR MED P.AV: 92620
MALDONAD.000 ES CAR MED P.AV: 92730
MELO.000 ES CAR MED P.AV: 92450
MERCEDES.000 ES CAR MED P.AV: 92640
MVA150.000 ES CAR MED P.AV: 92100
MVB150.000 ES CAR MED P.AV: 90200
MVC150.000 ES CAR MED P.AV: 92210
MVE150.000 ES CAR MED P.AV: 92240
MVF150.000 ES CAR MED P.AV: 92310
MVG150.000 ES CAR MED P.AV: 92320
MVH150.000 ES CAR MED P.AV: 92330
MVJ150.000 ES CAR MED P.AV: 92270
MVK150.000 ES CAR MED P.AV: 92340
MVM150.000 ES CAR MED P.AV: 92180
MVR150.000 ES CAR MED P.AV: 92350
NORTE.000 ES CAR MED P.AV: 9211

In [48]:
create_dataset(datos_path+'Carga Activa.csv', diccionario, datos_path + 'LOADP_total_psse.csv')

C:\Users\santi\AppData\Local\Temp\ipykernel_33412\677009404.py:2: DtypeWarning: Columns (3,4,11,15,16,48,50,58,64,66) have mixed types. Specify dtype option on import or set low_memory=False.
  original_data = pd.read_csv(path)
Cleaning data: 100%|██████████| 52/52 [02:17<00:00,  2.64s/it]


,92670,92060,92090,92710,92540,92600,92690,92940,92531,92681,...,92380,92470,92630,92410,92070,92590,92430,92420,92020,Fecha
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fecha
1,9.54016,2.8500,9.809999,31.236240,4.399368,11.598300,0.587500,0.000,17.590748,4.980417,...,41.310001,0.0315,20.700001,17.437859,2.056735,5.57550,31.130756,6.30943,0.000000,2021-03-21 00:00:00
2,9.54016,2.8500,9.809999,31.459021,4.399368,11.598300,0.587500,0.000,17.590748,4.980417,...,40.770000,0.0315,20.160000,17.437859,2.056735,5.57550,31.130756,5.79276,0.000000,2021-03-21 00:10:00
3,9.54016,2.8500,9.270000,30.797832,4.399368,11.598300,0.587500,0.000,17.040976,4.980417,...,40.049999,0.0315,20.160000,17.451233,2.056735,5.05386,30.539799,5.79276,0.000000,2021-03-21 00:20:00
4,9.54016,2.8500,9.270000,30.599020,4.399368,11.598300,0.587500,0.000,16.520864,4.980417,...,39.509998,0.0315,19.420000,16.903900,2.056735,5.05386,29.995466,5.79276,0.000000,2021-03-21 00:30:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135500,0.00000,2.8809,8.794800,29.803999,0.452673,18.341820,0.004167,6.066,18.642984,9.855135,...,53.820000,0.0240,21.900000,9.614657,6.041055,6.81912,30.337517,6.41386,9.182652,2023-10-17 23:10:00
135501,0.00000,2.8809,9.306000,30.353001,0.452673,18.341820,0.004167,6.066,19.722120,10.275838,...,53.820000,0.0240,21.900000,9.719620,5.550055,6.81912,30.854441,6.41386,9.182652,2023-10-17 23:20:00
135502,0.00000,2.8809,9.306000,29.153999,0.452673,18.850861,0.004167,6.066,19.722120,8.710971,...,53.820000,0.0240,21.900000,9.719620,6.222254,6.81912,30.854441,6.41386,9.182652,2023-10-17 23:30:00
135503,0.00000,2.3802,9.306000,28.695000,0.452673,18.338039,0.004167,6.066,19.203396,10.263849,...,52.560001,0.0240,21.900000,9.059999,6.036455,6.31260,30.854441,6.41386,9.182652,2023-10-17 23:40:00


In [49]:
entradas = [
    # (92670, None),
    (92060, "ARAPEY.150 TR TR1 MED Q.AV"),
    (92090, "ARTIGAS.150 TR TR1 MED Q.AV"),
    # (92710, None),
    # (92540, None),
    # (92600, None),
    # (92690, None),
    (92940, "DOLORES.150 TR TR1 MED Q.AV"),
    (92531, "DURAZNO.150 TR TR3 MED Q.AV"),
    (92681, "EFICE.150 TR TR1 MED Q.AV"),
    # (92440, None),
    # (92050, None),
    # (92520, None),
    # (92620, None),
    # (92730, None),
    (92450, "MELO.150 TR TR3 MED Q.AV"),
    # (92640, None),
    (92100, "MVA150.150 TR TR1 MED Q.AV"),
    (90200, "MVB150.030 TR TR1 MED Q.AV"),
    (92210, "MVC150.030 TR TR1 MED Q.AV"),
    (92240, "MVE150.010 TR TR3 MED Q.AV"),
    (92310, "MVF150.022 TR TR3 MED Q.AV"),
    (92320, "MVG150.022 TR TR1 MED Q.AV"),
    (92330, "MVH150.030 TR TR3 MED Q.AV"),
    (92270, "MVJ150.030 TR TR2 MED Q.AV"),
    (92340, "MVK150.030 TR TR1 MED Q.AV"),
    (92180, "MVM150.030 TR TR1 MED Q.AV"),
    (92350, "MVR150.022 TR TR1 MED Q.AV"),
    (92110, "NORTE.150 TR TR1 MED Q.AV"),
    (92650, "NPALMIRA.150 TR TR1 MED Q.AV"),
    (92500, "PAL150.030 TR TR1 MED Q.AV"),
    (92770, "PANDO.150 TR TR3 MED Q.AV"),
    (92030, "PAYSANDU.150 TR TR3 MED Q.AV"),
    # (92721, None),
    # (101, None),
    (92280, "PIEDRAS.150 TR TR3 MED Q.AV"),
    # (92400, None),
    # (92760, None),
    (92580, "RODRIGUE.150 L1 TRI MED Q.AV"),
    (92610, "ROSARIO.150 TR TR1 MED Q.AV"),
    (92040, "SALTO.150 TR TR1 MED Q.AV"),
    (92700, "SCA150.030 TR TR1 MED Q.AV"),
    (92380, "SOLYMAR.150 TR TR3 MED Q.AV"),
    # (92470, None),
    (92630, "SVAZQUEZ.150 TR TR3 MED Q.AV"),
    (92410, "TAC150.030 TR TR1 MED Q.AV"),
    (92070, "TGOMENSO.150 TR TR2 MED Q.AV"),
    (92590, "TRINIDAD.150 TR TR1 MED Q.AV"),
    (92430, "TYTRES.150 TR TR3 MED Q.AV"),
    (92420, "VALENTIN.150 TR TR2 MED Q.AV"),
    (92020, "YOUNG.150 TR TR2 MED Q.AV")
]

# Creación del diccionario con manejo de claves duplicadas
diccionario = {j:i for i,j in entradas}

diccionario['Fecha'] = 'Fecha'
# Mostrar el diccionario resultante
for k, v in diccionario.items():
    print(f"{k}: {v}") 

ARAPEY.150 TR TR1 MED Q.AV: 92060
ARTIGAS.150 TR TR1 MED Q.AV: 92090
DOLORES.150 TR TR1 MED Q.AV: 92940
DURAZNO.150 TR TR3 MED Q.AV: 92531
EFICE.150 TR TR1 MED Q.AV: 92681
MELO.150 TR TR3 MED Q.AV: 92450
MVA150.150 TR TR1 MED Q.AV: 92100
MVB150.030 TR TR1 MED Q.AV: 90200
MVC150.030 TR TR1 MED Q.AV: 92210
MVE150.010 TR TR3 MED Q.AV: 92240
MVF150.022 TR TR3 MED Q.AV: 92310
MVG150.022 TR TR1 MED Q.AV: 92320
MVH150.030 TR TR3 MED Q.AV: 92330
MVJ150.030 TR TR2 MED Q.AV: 92270
MVK150.030 TR TR1 MED Q.AV: 92340
MVM150.030 TR TR1 MED Q.AV: 92180
MVR150.022 TR TR1 MED Q.AV: 92350
NORTE.150 TR TR1 MED Q.AV: 92110
NPALMIRA.150 TR TR1 MED Q.AV: 92650
PAL150.030 TR TR1 MED Q.AV: 92500
PANDO.150 TR TR3 MED Q.AV: 92770
PAYSANDU.150 TR TR3 MED Q.AV: 92030
PIEDRAS.150 TR TR3 MED Q.AV: 92280
RODRIGUE.150 L1 TRI MED Q.AV: 92580
ROSARIO.150 TR TR1 MED Q.AV: 92610
SALTO.150 TR TR1 MED Q.AV: 92040
SCA150.030 TR TR1 MED Q.AV: 92700
SOLYMAR.150 TR TR3 MED Q.AV: 92380
SVAZQUEZ.150 TR TR3 MED Q.AV: 92630
TAC150

In [50]:
create_dataset(datos_path+'Carga Reactiva.csv', diccionario, datos_path + 'LOADQ_total_psse.csv')

C:\Users\santi\AppData\Local\Temp\ipykernel_33412\677009404.py:2: DtypeWarning: Columns (5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148) have mixed types. Specify dtype option on import or set low_memory=False.
  original_data = pd.read_csv(path)
Cleaning data: 100%|██████████| 36/36 [00:43<00:00,  1.20s/it]


,92060,92090,92940,92531,92681,92450,92100,90200,92210,92240,...,92700,92380,92630,92410,92070,92590,92430,92420,92020,Fecha
0,0.00,0.0000,0.000,0.00,0.000,0.000,0.00,0.000000,0,0,...,0.000,0.000,0,0.00000,0.000,0.000,0.000,0.000,0.000,2021-03-21 00:00:00
1,0.00,0.0000,0.000,0.00,0.000,0.000,0.00,0.000000,0,0,...,0.000,0.000,0,0.00000,0.000,0.000,0.000,0.000,0.000,2021-03-21 00:10:00
2,0.00,0.0000,0.000,0.00,0.000,0.000,0.00,0.000000,0,0,...,0.000,0.000,0,0.00000,0.000,0.000,0.000,0.000,0.000,2021-03-21 00:20:00
3,0.00,0.0000,0.000,0.00,0.000,0.000,0.00,0.000000,0,0,...,0.000,0.000,0,0.00000,0.000,0.000,0.000,0.000,0.000,2021-03-21 00:30:00
4,0.00,0.0000,0.000,0.00,0.000,0.000,0.00,0.000000,0,0,...,0.000,0.000,0,0.00000,0.000,0.000,0.000,0.000,0.000,2021-03-21 00:40:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136795,-0.54,-1.6605,-0.414,0.61,0.576,-0.225,1.68,-0.300595,0,0,...,0.051,2.286,0,-0.51408,-0.012,0.405,0.564,0.255,-0.945,2023-10-26 23:10:00
136796,-0.54,-1.6605,-0.414,0.61,0.576,-0.225,1.68,-0.300595,0,0,...,0.051,1.758,0,-0.51408,-0.012,0.405,0.564,0.255,-0.945,2023-10-26 23:20:00
136797,-0.54,-1.6605,-0.414,0.61,0.456,-0.225,1.68,-0.300595,0,0,...,0.051,1.758,0,-0.51408,-0.012,0.405,0.564,0.255,-0.945,2023-10-26 23:30:00
136798,-0.54,-1.6605,-0.414,0.06,0.456,-0.225,1.68,-0.300595,0,0,...,0.051,1.758,0,-0.51408,-0.012,0.405,0.564,0.255,-0.945,2023-10-26 23:40:00
